In [1]:
print("hello world")

hello world


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq
model= ChatGroq(
    model="llama3-70b-8192",
    api_key=GROQ_API_KEY,
    temperature=0,
)

In [4]:
from langchain_community.utilities import SQLDatabase

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [6]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(model, db)

response = chain.invoke({"question": "How many species of trees are in San Francisco?"})

print(response)

Here is the answer:

Question: How many species of trees are in San Francisco?
SQLQuery: SELECT COUNT(DISTINCT "qSpecies") FROM street_trees;


In [7]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [9]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(model, db)

execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query



C:\Users\DELL\AppData\Local\Temp\ipykernel_9048\3605226503.py:5: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [10]:
chain.invoke({"question": "List the species of trees that are present in San Francisco"})

'Error: (sqlite3.OperationalError) near "Here": syntax error\n[SQL: Here is the answer:\n\nQuestion: List the species of trees that are present in San Francisco\nSQLQuery: SELECT DISTINCT "qSpecies" FROM street_trees LIMIT 5;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'